<a href="https://colab.research.google.com/github/PedroQueiruga/Projeto_ImersaoIA_Alura_Pedro/blob/main/Projeto_imers%C3%A3oIA_PedroGQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Seção 1: Configurações Iniciais

* Nessa primeira seção, vamos dedicar à configuração inicial do ambiente e à preparação das ferramentas que serão utilizadas posteriormente.

* O passo a passo do que acontece é:
  * Instalar os programas e bibliotecas extras que o
  restante do código vai precisar para funcionar;
  * Configurar a chave de acesso pessoal para acessar os serviços do Google AI;
  * Preparamos ferramentas do Google AI para serem utilizadas;
  * Importamos ferramentas que ajudaram a mostrar os resultados de forma mais fácil de visualizar no notebook.

In [1]:
#Vamos começar instalando as componentes importantes:

!pip -q install google-genai
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [2]:
#Configurando a API Key do Google

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
#Configurando a SDK (Software Development Kit) do Gemini

from google import genai

clint = genai.Client()

model = "gemini-2.0-flash"


In [4]:
#Colocando as componentes ADK (Agent Development Kit) de agentes no programa

from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date

#Os facilitadores para visualização, que vão formatar a saída de texto

import textwrap
from IPython.display import display, Markdown
import requests
import warnings

warnings.filterwarnings("ignore")



In [5]:
#Agora, vamos criar uma função auxiliar para chamar o agente:

def call_agent(agent: Agent, message_text:str) -> str:
  #Criar o serviço na memória
  session_service = InMemorySessionService()
  #Criar uma sessão nova do serviço
  session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
  #Criar um Runner para o agente (Responsável por gerenciar agentes e seu fluxo de trabalho)
  runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
  #Criar o conteúdo da mensagem de entrada
  content = types.Content(role="user", parts=[types.Part(text=message_text)])

  final_response = ""

  # Itera assincronamente pelos eventos retornados durante a execução do agente
  for event in runner.run(user_id="user1", session_id="session1", new_message=content):
      if event.is_final_response():
        for part in event.content.parts:
          if part.text is not None:
            final_response += part.text
            final_response += "\n"
  return final_response


In [6]:
#Por fim, vamos criar uma função auxiliar para exibição do resultado:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

## Seção 2: Definindo os Agentes

* Nesta seção, iremos definir os agentes que serão utilizados. Eles irão se comunicar entre si para fornecer nossa resposta final.
* Os agentes são:
  * Agente 1: Pesquisador
    * Tem como objetivo a partir do tópico de entrada realizar uma extensa pesquisa sobre o tema, limitando a até 5 artigos ou notícias relevantes relacionadas. Extrai as informações dos textos para passar para o próximo agente.
  * Agente 2: Simplificador e Interpretador
    * Atua pegando o resultado vindo do agente pesquisador e o processando, de modo a montar uma estrutura onde é claro os seguintes tópicos de cada uma das fotnes: O que é, como foi feito, o que descobriram, porque é importante, link de acesso original, quem descobriu, instituição de ensino, ano de publicação.
  * Agente 3: Gerador de Texto de Divulgação Científica
    * A partir das informações vindas do agente simplificador, esse último agente monta a estrutura do texto de divulgação científica. Seguindo uma estrutura de partes inicialmente, que são juntadas para formar um texto completo que a partir de uma revisão humana pode ser publicado em um blog ou revista de divulgação científica.
* Importante ter em mente que apesar da inteligência artificial fazer um bom trabalho coletando e processando informações, o trabalho humano é essencial para a partir das informações fornecidas montar um artigo de divulgação científica.




In [7]:
##########################################
# --- Agente 1: Pesquisador --- #
##########################################
def agente_pesquisador(topico):
  pesquisador = Agent(
      name = "agente_pesquisador",
      model = "gemini-2.0-flash",
      instruction="""
      Você é um pequisador que deseja produzir conteúdo de divulgação científica. A sua tarefa é
      receber um tema de pesquisa ou uma descrição e encontrar artigos acadêmicos, ou notícias
      relevantes. A sua tarefa é usar a ferramenta de busca do google (google_search), para extrair
      as informações mais relevantes sobre o tema. Selecione até 5 fontes relevantes para o tema, se possível
      e extraia as informações do tema e qual o nome do artigo original, quais são os pesquisadores envolvidos,
      qual é a instituição de ensino envolvida, qual é o ano de publicação, qual é o título do artigo, qual é o resumo do artigo,
      qual é o link do artigo.
      """,
      description="Agente que seleciona e extrai informações de artigos acadêmicos com potencial para divulgação.",
      tools=[google_search]
  )

  entrada_do_agente_pesquisador = f"Tópico{topico}\n"

  #Executar o agente
  resultado_pesquisador = call_agent(pesquisador, entrada_do_agente_pesquisador)

  return resultado_pesquisador


In [24]:
##########################################
# --- Agente 2: Simplificador e Interpretador --- #
##########################################
def agente_simplificador(topico, resultado_pesquisador):
    simplificador = Agent(
        name="agente_simplificador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um divulgador científico especializado em simplificar conceitos complexos
        para um público amplo. Você receberá informações sobre artigos acadêmicos, incluindo
        seus resumos. Sua tarefa é ler cada resumo, artigo e matéria de notícias e simplificar
        o conteúdo para que possa ser facilmente comprendido por uma pessoa fora da área.

        Evite termos técnicos sempre que possivel, ou os explique brevemente. Apresente o conteúdo de cada
        fonte dando enfoque para os seguintes pontos: "O que foi feito", "O que foi descoberto"
        e "por que isso é importante para nós".

        Artigo 1 Simplificado:
        O que é: [Explicação simples do tema/problema]
        Como foi feito: [Descrição simplificada da metodologia]
        O que descobriram: [Resultados principais em linguagem acessível]
        Por que é importante: [Explicação do impacto/relevância]
        link de acesso: [Link do artigo original ou do site onde as informações foram retiradas, caso o link do artigo original não seja encontrado]
        Quem descobriu: [Nome do pesquisador]
        Instituição de ensino: [Nome da instituição]
        Ano de publicação: [Ano de publicação]

        Artigo 2 Simplificado:
        ...e assim por diante.

        """,
        description="Agente pesquisador, responsável por simplificar o conteúdo técnico de artigos acadêmicos.",

    )

    entrada_simplificador = f"Tópico:{topico}\n\nInformações dos Artigos:\n{resultado_pesquisador}"

    resultado_simplificador = call_agent(simplificador, entrada_simplificador)

    return resultado_simplificador

In [25]:
##########################################
# --- Agente 3: Gerador de Texto de Divulgação científica --- #
##########################################
def agente_gerador(topico,resultado_simplificador):
  gerador = Agent(
      name = "agente_gerador",
      model = "gemini-2.0-flash",
      instruction="""
      Você é um escritor de divulgação científica. Você receberá informações simplificadas
      sobre artigos acadêmicos. Sua tarefa é copilar este conteúdo em um texto coeso e
      envolvente que possa ser publicado em um site ou artigo de divulgação científica.
      O formato deve ser adequado para um públicoa amplo, incluindo pessoas sem formação
      científica.

      **Etapa 1: Estruturação do conteúdo em partes principais (Não apresentar ao usuário)**

      Separe as partes do texto inicialmente da seguinte forma:
      -Título curto, chamativo e representativo do texto noticiado
      -Subtítulo que complete a informação
      -Lide: Apresenta de forma resumida o conteúdo a ser noticiado, elaborando brevemente o tema com os
      questionamentos, o que será reportado? Quem identificou ou propos o que será discutido? Onde ou como
      foi desenvolvido o estudo? Quando foi desenvolvido? Como? Por quê? Quais os resultados? Quais as implicações?
      -Desenvolvimento Inicial: Pontuar os elementos imprescindíveis que o leitor tome conhecimento para o
      entendimento do texto, elaborando aspectos teóricos e metodológicos que sustentam a investigação do tema. Use um ou mais parágrafos.
      -Desenvolvimento Final: Apresentar os resultados obtidos, destacando os principais achados e suas implicações. Use um ou mais parágrafos.
      -Finalização do texto: Conclusão do texto, destacando a importância do estudo e possíveis avanços futuros.Use um ou mais parágrafos.
      -Referências: liste as fontes simplificadas recebidas, incluindo o nome/título e o link de acesso original. Caso não seja possível
      encontrar o link original do artigo, pode colocar o da notícia ou local de onde foi retirado.


      **Etapa 2: Gerar o Texto Corrido (Apresentar ao usuário)**

      Em seguida, pegue o conteúdo que você estruturou na Etapa 1 e escreva um texto contínuo,
      fluindo naturalmente entre as partes em formato de parágrafos. Siga a ordem
      definida na Etapa 1: Título, Subtítulo, Lide, Desenvolvimento Inicial, Desenvolvimento Final, Finalização e Referências.

      Mantenha um tom entusiasmado e acessível. Use transições suaves entre os parágrafos.
      O objetivo é criar uma narrativa clara que seja fácil e agradável de ler para um público sem conhecimento prévio na área.


      Evite jargões e explicações excessivamente complexas.
      O objetivo é informar, educar e inspirar o interesse pela ciência.
      """,
      description='Agente responsável por desenvolver o texto de divulgação científica'
  )

  entrada_gerador = f"Tópico:{topico}\n\nInformações dos Artigos Simplificados:\n{resultado_simplificador}"

  resultado_gerador = call_agent(gerador, entrada_gerador)

  return resultado_gerador

### Seção 3: Resultados

* Agora vamos ao resultado final, o tópico será fornecido pelo usuário e a partir disso, um texto de divulgação científica de resultados recentes na área de pesquisa desejada será montado.
* Extremamente útil para descrever o estado da arte atual de uma área de pesquisa, quais suas aplicações e importância.


In [ ]:
#Agora, vamos produzir os resultados

#Obtendo tópico do usuário:

topico = input("Digite o tópico que deseja pesquisar: ")

#Inserindo lógica do sistema de agentes
if not topico:
  print("Você esqueceu de digitar o tópico!")
else:
  print(f"Maravilha! Vamos escrever o texto!")

  #Agora, vamos colocar os agentes para trabalhar:

  resultado_pesquisador = agente_pesquisador(topico)
  print("Resultado do agente pesquisador:\n")
  display(to_markdown(resultado_pesquisador))

  resultado_simplificador = agente_simplificador(topico, resultado_pesquisador)
  print("Resultado do agente simplificador:\n")
  display(to_markdown(resultado_simplificador))

  resultado_gerador = agente_gerador(topico, resultado_simplificador)
  print("Resultado do agente gerador:\n")
  display(to_markdown(resultado_gerador))